1\. Write a function that converts number representation (bin<->dec<->hex)

In [12]:
def conv(num):
    if isinstance(num, str): #if num is a string then the number can be binary or hexadecimal
        try:
            n_dec = int(num, 2) #check if the num is binary
            print("Decimal representation of", num, "is", n_dec)
            print("Hexadecimal representation of", num, "is", hex(n_dec))
        except:
            n_dec = int(num,16) #check if the num is hex
            print("Decimal representation of", num, "is", n_dec)
            print("Binary representation of", num, "is", bin(n_dec))
    else: #if num is not a string the number is in decimal representation
        print("Binary representation of", num, "is", bin(num))
        print("Hexadecimal representation of", num, "is", hex(num))
    return
          
conv("4f")

Decimal representation of 4f is 79
Binary representation of 4f is 0b1001111


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [41]:
def floatp(number):
    if len(number) == 32:
        sign = int(number[0])
        exp = int(number[1:9],2)
        mantissa = number[9:]
        s = 0
        i = 1
        for b in mantissa:
            s += int(b)*2**(-i)
            i += 1
            value = (-1)**(sign)*2**(exp-127)*(1+s)
    return value

floatp("01000011111011100000000000000000")

476.0

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [14]:
over = 1
overflow = False
while overflow == False:
    try:
        over*=2
        overflow = False
    except:
        overflow = True
print(over)


...


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [10]:
for e in [14,15,16]: print (7+1.0*10**-e)

7.00000000000001
7.000000000000001
7.0


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [41]:
import math
def sol(a, b, c):
    x1 = (-b + math.sqrt(b**2 - 4*a*c))/(2*a)
    x2 = (-b - math.sqrt(b**2 - 4*a*c))/(2*a)
    return x1, x2
print(sol(0.001, 1000, 0.001))

def sol_1(a,b,c):
    x1 = (4*a*c)/(2*a*(-b-math.sqrt(b**2 - 4*a*c)))
    x2 = (4*a*c)/(2*a*(-b+math.sqrt(b**2 - 4*a*c)))
    return x1,x2
print(sol_1(0.001, 1000, 0.001))

(-9.999894245993346e-07, -999999.999999)
(-1.0000000000009999e-06, -1000010.5755125057)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [51]:
def f(x):
    return x*(x-1)

def der_f(x, delta):
    der = (f(x+delta)-f(x))/delta
    return der
der_f(1, 10**(-12))

1.0000889005833413

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [30]:
import numpy as np
import math

def integral(N):
    width = 2/N
    sum = 0
    for i in np.arange(-1+width, 1-width, width):
        sum += width*math.sqrt(1 - i**2)
    return sum
integral(100)


1.5651543058008217